CSC535 - Data Mining
HW2

In [21]:
import pandas as pd
import numpy as np
from math import log
from collections import OrderedDict

#Part 1: Implement the ID3 decision tree induciton algorithm 

#create ID3 fucntion
'''def ID3(data, label):
input parameters: data, label
creates a rootNode
creates a tree
returns: tree
'''
def ID3(data, label):
    rootNode = getRootNode(data, label)
    tree = (rootNode, getAttribute(data, label, rootNode))
    return tree

def getAttribute(data, label, rootNode):
    gets a list of attributes under each attribute and recursively creates a sub tree
    input parameters: data, label, rootnode
    returns: a dictionary list of attributes

In [22]:
'''def getAttribute(data, label, rootNode):
gets a list of attributes under each attribute and recursively creates a sub tree
input parameters: data, label, rootnode
returns: a dictionary list of attributes
'''
def getAttribute(data, label, rootNode):
    tree = {}
    uniqueValues = data[rootNode].unique()
    defaultData = data[label].value_counts().idxmax()
    tree.update({'default': defaultData})
    for uniqueValue in uniqueValues:
        dataf = data[data[rootNode] == uniqueValue]
        info = calcInfo(dataf[label])
        if info > 0:
            dataf = dataf.drop([rootNode], axis=1)
            nodeA = getRootNode(dataf, label)
            tupl = (nodeA, getAttribute(dataf, label, nodeA))
            tree.update({str(uniqueValue): tupl})
        else:
            d = dataf.filter(items=[label, rootNode]).values[0]
            key = str(d[1])
            value = d[0]
            tree.update({key: value})
    return dict(OrderedDict(sorted(tree.items(), key=lambda d: d[0])))

In [23]:
'''def getRootNode(data, label):
gets root node for spec attribute list
returns: root node
'''
def getRootNode(data, label):
    gainDict = dict()
    info = calcInfo(data[label])
    if info > 0:
        for key in data.keys():
            if key != label:
                dataf = data.filter(items=[label, key])
                gainDict.update({key: calcGain(dataf, key, label, info)})
    root = max(gainDict.keys(), key=(lambda x: gainDict[x]))
    return root

In [24]:
'''
def calcGain(data, key, label, iTotal):
    calculates information gain value for speciric node
'''

def calcGain(data, key, label, iTotal):
    data = pd.DataFrame(data=data)
    entropyTotal = 0.0
    uniqueValues = data[key].unique()
    for value in uniqueValues:
        dataf = data[data[key] == value]
        infoValue = calcInfo(data[label])
        entropyValue = calcEntropy(s=len(dataf), totalS=len(data), info=infoValue)
        entropyTotal += entropyValue
    return float(format((iTotal - entropyTotal), '.5f'))

In [25]:
'''def calcEntropy(s, totalS, info):
calculates entropy value for specific node
'''
def calcEntropy(s, totalS, info):
    s = abs(s)
    totalS = abs(totalS)
    if s != 0 and totalS != 0:
        result = (s / totalS) * info
    else:
        result = 0
    return float(format(result, '.5f'))

In [26]:
'''
def calcInfo(data):
    calculations information value for specific noe
'''
def calcInfo(data):
    info = 0.0
    uniqueValues = data.value_counts()
    for count in uniqueValues:
        p = calcProbability(count, len(data))
        if p != 0:
            temp1 = p * (log(p, 2))
        else:
            temp1 = 0
        info -= temp1
    return float(format(info, '.5f'))

In [27]:
'''def calcProbability(s1, s):
calculates probability
'''

def calcProbability(s1, s):
    s1 = abs(s1)
    s = abs(s)
    if s1 != 0 and s != 0:
        result = s1 / s
    else:
        result = 0
    return float(format(result, '.5f'))

In [28]:
'''classifies input from decision tree and returns class value
'''
def classify(data, input):
    if isinstance(data, tuple):
        if data[0] in input:
            attributeData = input[data[0]]
            if attributeData in data[1]:
                value = data[1].get(attributeData)
                result = classify(value, input)
            else:
                value = data[1].get('default')
                result = classify(value, input)
        else:
            value = data[1].get('default')
            result = classify(value, input)
    else:
        result = data
    return result

In [29]:
''' creates dataframe called data2 for training data and class label
'''
def dataPreprocesing(data, label):
    dicList = list()
    for row in data:
        temp = {label: bool(row[1])}
        row[0].update(temp)
        dicList.append(row[0])
    data2 = pd.DataFrame(data=dicList)
    
    return data2

In [30]:
'''gives the user the ability to read in a file if needed'''
def readDataset(filename):
    dataframe = pd.read_table(filename, sep=',')
    return dataframe

In [31]:
training_data = [
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, False)
]

label = 'hired'
dataf = dataPreprocesing(training_data, label)
dt = ID3(dataf, label)

print('DT= \n', dt, '\n')
c1 = {"level": "Junior", "lang": "Java", "tweets": "yes", "phd": "no"}
c2 = {"level": "Junior", "lang": "Java", "tweets": "yes", "phd": "yes"}
c3 = {"level": "Intern"}
c4 = {"level": "Senior"}
c1_result = classify(dt, c1)
c2_result =classify(dt, c2)
c3_result =classify(dt, c3)
c4_result =classify(dt, c4)
print("Classify1 =", c1,":", c1_result)
print("Classify2 =", c2,":", c2_result)
print("Classify3 =", c3,":", c3_result)
print("Classify4 =", c4 ,":", c4_result)


DT= 
 ('level', {'Junior': ('lang', {'Python': ('tweets', {'default': True, 'no': ('phd', {'default': False, 'no': True, 'yes': False}), 'yes': True}), 'R': ('tweets', {'default': False, 'yes': ('phd', {'default': False, 'no': True, 'yes': False})}), 'default': True}), 'Mid': True, 'Senior': ('lang', {'Java': False, 'Python': ('tweets', {'default': False, 'no': False, 'yes': True}), 'R': True, 'default': False}), 'default': True}) 

Classify1 = {'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'} : True
Classify2 = {'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd': 'yes'} : True
Classify3 = {'level': 'Intern'} : True
Classify4 = {'level': 'Senior'} : False


In [32]:
#Part 2: Find a real data set. Build a DT model for this data set and test your model. 
#Section 2
#data source: https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset/version/290
#my csv file for this dataset was too large. I had to edit it and try to port it in. It did not work, so I 
#created my own training_data set for this. This section does not work.

training_data1 = [
    ({'date': '1/21/2020', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/22/2020', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/23/2020', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/24/2020', 'county': 'Cook', 'state': 'Illinois', 'fips': '17031', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/24/2020', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/25/2020', 'county': 'Orange', 'state': 'Califrornia', 'fips': '6059', 'cases': '1', 'deaths':'1'}),
    ({'date': '1/25/2020', 'county': 'Snohomish', 'state': 'Washington', 'fips': '53061', 'cases': '1', 'deaths':'1'}),
]
label = 'result'
label = 'result'
dataf = dataPreprocesing(training_data1, label)
dt1 = ID3(training_data1, label)


print('DT= \n', dt1, '\n')


KeyError: 1